<a href="https://colab.research.google.com/github/LubichSG/Thinking-structurally/blob/master/Bithday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia-api
!pip install convertdate
!pip install holidays
!pip install astral
!pip install timezonefinder
!pip install geopy
import wikipediaapi
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from convertdate import hebrew
import holidays
import calendar
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim
from astral import LocationInfo
from astral.sun import sun

Этот код анализирует дату рождения человека, проводя следующие действия:

Получение часового пояса и времени заката: Функция get_timezone_and_sunset() получает географическое положение города, используя библиотеку geopy и определяет часовой пояс для заданных координат. Затем она вычисляет время заката для указанного города на основе местного времени.

Конвертация даты из григорианского календаря в еврейский: Функция gregorian_to_hebrew() преобразует дату из григорианского календаря в еврейскую. Если время дня больше 18:00, она сдвигает дату на следующий день в соответствии с еврейским календарем (еврейские сутки начинаются вечером).

Определение еврейского праздника: Функция is_jewish_holiday() проверяет, попадает ли данная дата на еврейский праздник, используя библиотеку праздников для Израиля.

Получение известных людей, родившихся в этот день: Функция get_births_on_date() использует API Википедии для получения списка известных людей, родившихся в заданный день. Она ищет информацию на английской странице Википедии по данной дате.

Анализ даты рождения: Функция analyze_birth_date() обрабатывает все данные, возвращая информацию о дате рождения:

Дата рождения в григорианском календаре
Время рождения
Место рождения
Дата в еврейском календаре
Соответствует ли дата еврейскому празднику
Ссылка на статью в Википедии с информацией о рождённых в этот день
Список известных людей, рождённых в этот день.
Взаимодействие с пользователем: В основной части кода пользователь вводит свою дату и время рождения, страну и город. Программа анализирует данные и выводит результат, используя функции для расчёта еврейской даты, поиска праздников и известных людей.

In [ ]:


# Функция для получения часового пояса и времени заката
def get_timezone_and_sunset(country, city):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(f"{city}, {country}")

    if location is None:
        raise ValueError(f"Unable to find coordinates for {city}, {country}")

    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lng=location.longitude, lat=location.latitude)

    loc = LocationInfo(name=city, region=country, timezone=timezone_str,
                       latitude=location.latitude, longitude=location.longitude)
    s = sun(loc.observer, date=datetime.now(ZoneInfo(timezone_str)))

    return timezone_str, s["sunset"].time()  # Возвращаем строку часового пояса и время заката (только время)

# Обновленная функция для конвертации григорианской даты в еврейскую
def gregorian_to_hebrew(year, month, day, hour, minute, sunset_time):
    # Конвертируем григорианскую дату в еврейскую
    heb_year, heb_month, heb_day = hebrew.from_gregorian(year, month, day)

    # Проверяем, нужно ли сдвинуть дату на следующий день
    current_time = datetime(year, month, day, hour, minute).time()

    if current_time >= sunset_time:  # Если время после заката, сдвигаем день
        # Используем datetime для добавления одного дня
        next_day = datetime(year, month, day) + timedelta(days=1)
        heb_year, heb_month, heb_day = hebrew.from_gregorian(next_day.year, next_day.month, next_day.day)

    heb_month_name = hebrew.MONTHS[heb_month - 1]
    return heb_day, heb_month_name, heb_year

def is_jewish_holiday(year, month, day):
    il_holidays = holidays.CountryHoliday('IL', years=year)
    date_to_check = datetime(year, month, day).date()
    return il_holidays.get(date_to_check)

def get_births_on_date(month, day):
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        user_agent='MyWikipediaApp (https://webanalyst.info; tsurevdan@gmail.com)'
    )

    month_name = calendar.month_name[month]
    page_title = f"{month_name}_{day}"

    page = wiki_wiki.page(page_title)

    if not page.exists():
        return f"Page for date {month_name} {day} not found.", ""

    births_section = next((section for section in page.sections if section.title.lower() == "births"), None)

    if not births_section:
        return f"Birth information for date {month_name} {day} not found.", ""

    births_text = ""
    count = 0  # Счетчик для ограничения количества записей
    for subsection in births_section.sections:
        if count >= 10:  # Останавливаем, если достигнуто 10 записей
            break
        births_text += f"{subsection.title}:\n{subsection.text}\n\n"
        count += 1

    article_link = f"Wikipedia article: {page.fullurl}"

    return births_text, article_link



def analyze_birth_date(year, month, day, hour, minute, country, city):
    # Получаем часовой пояс и время заката
    timezone_str, sunset = get_timezone_and_sunset(country, city)
    birth_datetime = datetime(year, month, day, hour, minute, tzinfo=ZoneInfo(timezone_str))

    # Конвертируем дату в еврейскую с учётом времени заката
    heb_day1, heb_month, heb_year = gregorian_to_hebrew(year, month, day, hour, minute, sunset)
    hebrew_date = f"{heb_day1} {heb_month} {heb_year}"

    # Проверяем, является ли это еврейским праздником
    holiday = is_jewish_holiday(year, month, day)

    # Получаем информацию о рождённых в этот день людях
    notable_births, article_link = get_births_on_date(month, day)

    return {
        "Gregorian calendar date": f"{day}.{month}.{year}",
        "Birth time": f"{hour:02d}:{minute:02d}",
        "Birth place": f"{city}, {country}",
        "Hebrew calendar date": hebrew_date,
        "Jewish holiday": holiday if holiday else "None",
        "Wikipedia article link": article_link,
        f"Born on {calendar.month_name[month]} {day}": notable_births,
    }
# Основной код
year = int(input("Enter birth year: "))
month = int(input("Enter birth month (1-12): "))
day = int(input("Enter birth day: "))
time_str = input("Enter birth time (HH:MM): ")
hour, minute = map(int, time_str.split(':'))
country = input("Enter birth country: ")
city = input("Enter birth city: ")

try:
    result = analyze_birth_date(year, month, day, hour, minute, country, city)
    for key, value in result.items():
        if key.startswith("Born on"):
            print(f"\n{key}:")
            print(value)
        else:
            print(f"{key}: {value}")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from collections import Counter

def get_weekday(year, month, day):
    return datetime(year, month, day).strftime("%A")

start_year = 1986
end_year = 2027
birthday_weekdays = []

for year in range(start_year, end_year + 1):
    weekday = get_weekday(year, 10, 4)  # 4 октября каждого года
    birthday_weekdays.append(weekday)

weekday_counts = Counter(birthday_weekdays)

print("Распределение дней недели для дня рождения (4 октября) :")
for day, count in weekday_counts.most_common():
    print(f"{day}: {count} раз")

print(f"\nНаиболее частый день недели: {weekday_counts.most_common(1)[0][0]}")

# Определение дней рождения, выпадающих на Рош ха-Шана
rosh_hashanah_birthdays = [
    1986,  # Возраст 0
    2005   # Возраст 19
]

print("\nГоды, когда день рождения совпадает с Рош ха-Шана:")
for year in rosh_hashanah_birthdays:
    weekday = get_weekday(year, 10, 4)
    print(f"{year}: {weekday}")

In [ ]:
from convertdate import hebrew
from datetime import datetime, date, timedelta
import holidays
import matplotlib.pyplot as plt

hebrew_months = [
    "Nisan", "Iyar", "Sivan", "Tammuz", "Av", "Elul",
    "Tishrei", "Cheshvan", "Kislev", "Tevet", "Shvat", "Adar", "Adar II"
]

def convert_to_hebrew_date(gregorian_date):
    year, month, day = gregorian_date.year, gregorian_date.month, gregorian_date.day
    return hebrew.from_gregorian(year, month, day)

def find_rosh_hashanah(year):
    for month in [9, 10]:
        for day in range(1, 31):
            try:
                greg_date = date(year, month, day)
                heb_date = hebrew.from_gregorian(year, month, day)
                if heb_date[1] == 7 and heb_date[2] == 1:
                    return greg_date
            except ValueError:
                continue
    return None

def calculate_days_difference(birthday, rosh_hashanah_date):
    birthday_that_year = date(rosh_hashanah_date.year, birthday.month, birthday.day)
    difference = (rosh_hashanah_date - birthday_that_year).days
    if difference < -180:
        return difference + 365
    elif difference > 180:
        return difference - 365
    else:
        return difference

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

def plot_difference(birthday, years_to_plot):
    ages = range(years_to_plot)
    rosh_hashanah_dates = []
    differences = []
    intersections = []

    for age in ages:
        year = birthday.year + age
        rosh_hashanah_date = find_rosh_hashanah(year)
        if rosh_hashanah_date:
            difference = calculate_days_difference(birthday, rosh_hashanah_date)
            rosh_hashanah_dates.append(rosh_hashanah_date)
            differences.append(difference)
            # Check if the dates are within 3 days of each other
            if abs(difference) <= 3:
                intersections.append((age, difference))
        else:
            rosh_hashanah_dates.append(None)
            differences.append(None)

    fig, ax = plt.subplots(figsize=(14, 7))

    valid_ages = [age for age, diff in zip(ages, differences) if diff is not None]
    valid_differences = [diff for diff in differences if diff is not None]

    ax.plot(valid_ages, valid_differences, marker='o', linestyle='-', color='blue', label='Difference')
    ax.axhline(y=0, color='r', linestyle='--', label='Birthday')

    # Highlight intersections
    intersection_ages = [i[0] for i in intersections]
    intersection_diffs = [i[1] for i in intersections]
    ax.scatter(intersection_ages, intersection_diffs, color='red', s=100, zorder=5, label='Near Intersection')

    for age, diff in intersections:
        ax.annotate(f'Age {age}', (age, diff), xytext=(5, 5), textcoords='offset points')

    ax.set_xlabel('Age')
    ax.set_ylabel('Days Difference (Positive: Rosh Hashanah after Birthday)')
    ax.set_title(f'Days between Birthday  (Age 0-{years_to_plot-1})')

    plt.xticks(range(0, years_to_plot, 5))
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()

    plt.tight_layout()
    plt.show()

    return rosh_hashanah_dates, differences, intersections

# Main execution
birth_date = datetime(1986, 10, 4)
years_to_plot = 51

hebrew_birthday = convert_to_hebrew_date(birth_date)
hebrew_day, hebrew_month, hebrew_year = hebrew_birthday
hebrew_month_name = hebrew_months[hebrew_month - 1]

print(f"Your Hebrew birthdate: {hebrew_day} {hebrew_month_name} {hebrew_year}")

israel_holidays = holidays.Israel(years=2024)
specific_date = date(2024, 10, 4)
if specific_date in israel_holidays:
    print(f"{specific_date} is a holiday: {israel_holidays[specific_date]}")
else:
    print(f"{specific_date} is not a holiday.")

age = calculate_age(birth_date)
print(f"The person's age today is: {age} years")

rosh_hashanah_dates, differences, intersections = plot_difference(birth_date, years_to_plot)

print(f"\nDifference between Birthday  from age 0 to {years_to_plot-1}:")
for age, rosh_hashanah_date, difference in zip(range(years_to_plot), rosh_hashanah_dates, differences):
    if rosh_hashanah_date and difference is not None:
        print(f"Age {age}: Rosh Hashanah on {rosh_hashanah_date.strftime('%d %B %Y')}, "
              f"Difference: {difference} days")
    else:
        print(f"Age {age}: Data not available")

print("\nAges when Hebrew and Gregorian birthdays nearly intersect (within 3 days):")
for age, diff in intersections:
    print(f"Age {age}: Difference of {diff} days")

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, date
from convertdate import hebrew

# Функция для конвертации григорианской даты в еврейскую
def convert_to_hebrew_date(gregorian_date):
    year, month, day = gregorian_date.year, gregorian_date.month, gregorian_date.day
    return hebrew.from_gregorian(year, month, day)

# Функция для конвертации еврейской даты обратно в григорианскую для конкретного года
def hebrew_to_gregorian(gregorian_year, hebrew_month, hebrew_day):
    hebrew_year = hebrew.from_gregorian(gregorian_year, 1, 1)[0]  # Получаем соответствующий еврейский год
    return date(*hebrew.to_gregorian(hebrew_year, hebrew_month, hebrew_day))

# Вычисление разницы между григорианским днем рождения и соответствующей еврейской датой
def calculate_days_difference(birthday, hebrew_birthday_date):
    birthday_that_year = date(hebrew_birthday_date.year, birthday.month, birthday.day)
    difference = (hebrew_birthday_date - birthday_that_year).days
    return difference

# Построение графика разницы между григорианской датой рождения и еврейской датой рождения
def plot_difference(birthday, years_to_plot):
    ages = range(years_to_plot)
    differences = []
    hebrew_birthdays = []
    intersections = []

    hebrew_birthday_initial = convert_to_hebrew_date(birthday)
    hebrew_day, hebrew_month = hebrew_birthday_initial[2], hebrew_birthday_initial[1]

    for age in ages:
        year = birthday.year + age
        # Находим еврейскую дату рождения для каждого года
        hebrew_birthday_date = hebrew_to_gregorian(year, hebrew_month, hebrew_day)
        difference = calculate_days_difference(birthday, hebrew_birthday_date)
        hebrew_birthdays.append(hebrew_birthday_date)
        differences.append(difference)
        # Проверяем, если дни почти совпадают (разница <= 3 дней)
        if abs(difference) <= 3:
            intersections.append((age, difference))

    # Построение графика
    fig, ax = plt.subplots(figsize=(14, 7))

    valid_ages = [age for age, diff in zip(ages, differences) if diff is not None]
    valid_differences = [diff for diff in differences if diff is not None]

    ax.plot(valid_ages, valid_differences, marker='o', linestyle='-', color='blue', label='Difference')
    ax.axhline(y=0, color='r', linestyle='--', label='Birthday')

    # Подсветка точек, где даты почти совпадают
    intersection_ages = [i[0] for i in intersections]
    intersection_diffs = [i[1] for i in intersections]
    ax.scatter(intersection_ages, intersection_diffs, color='red', s=100, zorder=5, label='Near Intersection')

    for age, diff in intersections:
        ax.annotate(f'Age {age}', (age, diff), xytext=(5, 5), textcoords='offset points')

    ax.set_xlabel('Age')
    ax.set_ylabel('Days Difference (Positive: Hebrew Birthday after Gregorian)')
    ax.set_title(f'Days between Gregorian and Hebrew Birthday (Age 0-{years_to_plot-1})')

    plt.xticks(range(0, years_to_plot, 5))
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()

    plt.tight_layout()
    plt.show()

    return hebrew_birthdays, differences, intersections

# Основной код для ввода данных
def main():
    year = int(input("Enter birth year: "))
    month = int(input("Enter birth month (1-12): "))
    day = int(input("Enter birth day: "))
    time_str = input("Enter birth time (HH:MM): ")
    hour, minute = map(int, time_str.split(':'))
    country = input("Enter birth country: ")
    city = input("Enter birth city: ")

    birth_date = datetime(year, month, day, hour, minute)

    # Возраст на текущую дату
    today = date.today()
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    print(f"The person's age today is: {age} years")

    # Построение графика разницы в днях между григорианской датой рождения и еврейской датой рождения
    years_to_plot = 51  # Количество лет для анализа
    hebrew_birthdays, differences, intersections = plot_difference(birth_date, years_to_plot)

    # Вывод различий
    print(f"\nDifference between Birthday and Hebrew Birthday from age 0 to {years_to_plot-1}:")
    for age, hebrew_birthday_date, difference in zip(range(years_to_plot), hebrew_birthdays, differences):
        if difference is not None:
            print(f"Age {age}: Hebrew Birthday on {hebrew_birthday_date.strftime('%d %B %Y')}, "
                  f"Difference: {difference} days")
        else:
            print(f"Age {age}: Data not available")

    # Вывод возрастов, когда дни почти совпадают
    print("\nAges when Birthday and Hebrew Birthday nearly intersect (within 3 days):")
    for age, diff in intersections:
        print(f"Age {age}: Difference of {diff} days")

if __name__ == "__main__":
    main()
